# 机器学习工作流
1. 数据准备
2. 训练集生成
3. 算法训练、评估、选择
4. 部署和监控

从开始的数据源，到最终的机器学习系统：![ML workflow](MLworkflow.png)

# 1.数据准备

1. 完全理解项目目标。比较依赖domain knowledge 和经验。如之前的广告点击预测、股价预测。
2. 收集尽可能多、全的数据
3. 保证特征值的一致性，比如gender字段的Male、M
4. 缺失值处理

### missing data imputation：

In [1]:
import numpy as np
from sklearn.preprocessing import Imputer
data_origin = [[30, 100],
               [20, 50],
               [35, np.nan],
               [25, 80],
               [30, 70],
               [40, 60]]

均值填充：

In [3]:
imp_mean = Imputer(missing_values='NaN', strategy='mean')
imp_mean.fit(data_origin) # fit data_origin
data_mean_imp = imp_mean.transform(data_origin) # transform data_origin
print(data_mean_imp)

[[  30.  100.]
 [  20.   50.]
 [  35.   72.]
 [  25.   80.]
 [  30.   70.]
 [  40.   60.]]


中位数填充：

In [4]:
imp_median = Imputer(missing_values='NaN', strategy = 'median')
imp_median.fit(data_origin)
data_median_imp = imp_median.transform(data_origin)
print(data_median_imp)

[[  30.  100.]
 [  20.   50.]
 [  35.   70.]
 [  25.   80.]
 [  30.   70.]
 [  40.   60.]]


带缺失值的新样本进来：

In [5]:
new = [[20, np.nan],
       [30, np.nan],
       [np.nan, 70],
       [np.nan, np.nan]]
new_mean_imp = imp_mean.transform(new)
print(new_mean_imp)

[[ 20.  72.]
 [ 30.  72.]
 [ 30.  70.]
 [ 30.  72.]]


In [6]:
# Effects of discarding missing values and imputation
from sklearn import datasets
dataset = datasets.load_diabetes()
X_full, y = dataset.data, dataset.target


# Simulate a corrupted data set by adding 25% missing values
m, n = X_full.shape
m_missing = int(m * 0.25)
print(m, m_missing)

# Randomly select m_missing samples
np.random.seed(42)
missing_samples = np.array([True] * m_missing + [False] * (m - m_missing))
np.random.shuffle(missing_samples)

# For each missing sample, randomly select 1 out of n features
missing_features = np.random.randint(low=0, high=n, size=m_missing)
# Represent missing values by nan
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = np.nan


# Discard samples containing missing values
X_rm_missing = X_missing[~missing_samples, :]
y_rm_missing = y[~missing_samples]

# Estimate R^2 on the data set with missing samples removed
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_score
regressor = RandomForestRegressor(random_state=42, max_depth=10, n_estimators=100)
score_rm_missing = cross_val_score(regressor, X_rm_missing, y_rm_missing).mean()
print('Score with the data set with missing samples removed: {0:.2f}'.format(score_rm_missing))


# Imputation with mean value
imp_mean = Imputer(missing_values='NaN', strategy='mean')
X_mean_imp = imp_mean.fit_transform(X_missing)
# Estimate R^2 on the data set with missing samples removed
regressor = RandomForestRegressor(random_state=42, max_depth=10, n_estimators=100)
score_mean_imp = cross_val_score(regressor, X_mean_imp, y).mean()
print('Score with the data set with missing values replaced by mean: {0:.2f}'.format(score_mean_imp))


# Estimate R^2 on the full data set
regressor = RandomForestRegressor(random_state=42, max_depth=10, n_estimators=500)
score_full = cross_val_score(regressor, X_full, y).mean()
print('Score with the full data set: {0:.2f}'.format(score_full))


442 110
Score with the data set with missing samples removed: 0.39
Score with the data set with missing values replaced by mean: 0.42
Score with the full data set: 0.44


### 并不能保证填充策略总是比舍弃好，可以采用交叉验证来得到最佳策略。

# 2.训练集的生成
## 数据预处理和特征工程

数据预处理包括：类别特征编码、特征缩放、特征选择、降维。

如果特征是类别特征，是否需要编码看之后所用的算法。如朴素贝叶斯、树算法可以直接处理，其他算法通常需要编码才能处理。

因为特征生成的output是算法训练的input，因此2个阶段需要整体看待，而不是分离对待。

### 是否需要进行特征选择，如果是，怎么进行？基于L1正则的Logistic 和随机森林。特征选择的好处：
    
    * 减少预测模型的训练时间，因为冗余或不相关的特征被排除了；
    * 减少过拟合的发生；
    * 可能提高性能，因为预测模型用更重要的特征进行学习，这需要交叉验证，如下。
    
采用手写数字数据集，SVM

In [2]:
import numpy as np
from sklearn.datasets import load_digits 
dataset = load_digits()
X, y = dataset.data, dataset.target
print(X.shape)

(1797, 64)


#### 基于原数据集的accuracy：

In [3]:
from sklearn.svm import SVC
from sklearn.model_selection import cross_val_score # 交叉验证给出评分，默认3折
classifier = SVC(gamma=0.005)
score = cross_val_score(classifier, X, y).mean()  # 默认3折，返回3各评分，所以取mean
print('Score weith the original data set: {0:.2f}'.format(score))

Score weith the original data set: 0.88


#### 先用随机森林做特征选择，基于这些特征再用SVC，并估计accuracy

In [5]:
from sklearn.ensemble import RandomForestClassifier
random_forest = RandomForestClassifier(n_estimators=100, criterion='gini', n_jobs=-1)
random_forest.fit(X, y)

# 按特征重要程度排序
feature_sorted = np.argsort(random_forest.feature_importances_) 

# Select different number of top features
K = [10, 15, 25, 35, 45]
for k in K:
    top_K_features = feature_sorted[-k:] # 重要特征的索引
    X_k_selected = X[:, top_K_features]  # 切片
    # 分别估计不同的 k 下的accuracy
    classifier = SVC(gamma=0.005)
    score_K_features = cross_val_score(classifier, X_k_selected, y).mean()
    print("Score with the data set of top {0} features: {1:.2f}".format(k, score_K_features))

Score with the data set of top 10 features: 0.87
Score with the data set of top 15 features: 0.93
Score with the data set of top 25 features: 0.94
Score with the data set of top 35 features: 0.92
Score with the data set of top 45 features: 0.88


选最重要的25个特征能得到最高的accuracy

### 是否降维？如何降维？
降维的好处与特征选择相似。下面用PCA降维，构建不同维数的新数据集，训练并估计各accuracy：

In [10]:
from sklearn.decomposition import PCA

# Keep different number of top components
N = [10, 15, 25, 35, 45]
for n in N:
    pca = PCA(n_components=n)
    X_n_kept = pca.fit_transform(X)
    # 估计accuracy
    classifier = SVC(gamma=0.005)
    score_n_components = cross_val_score(classifier, X_n_kept, y).mean()
    print('Score with the data set of top {0} components: {1:.2f}'.format(n,score_n_components))

Score with the data set of top 10 components: 0.95
Score with the data set of top 15 components: 0.95
Score with the data set of top 25 components: 0.91
Score with the data set of top 35 components: 0.89
Score with the data set of top 45 components: 0.88


降维到10或15维，accuracy都能有0.95

## 特征缩放
股价预测案例中，SGD线性回归和SVR要求特征标准化-移除平均并缩放到单位方差。那么什么适合需要特征缩放？

通常，朴素贝叶斯和树算法对特征量纲的不同并不敏感，因为它们在乎特征的独立性。Logistic 或线性回归一般也不受影响，**除非w的优化方式是SGD**。

大部分情况下，一个算法涉及到样本距离 就需要 缩放／标准化 features，比如SVC、SVR。任何使用SGD的算法也必须要做特征缩放。

## 特征工程

有领域知识，再好不过；若无领域知识，如何生成新特征？

### Binarization:
设定一个阈值，将数值特征转换为'二元'特征。例如，垃圾邮件检测，特征（term） 'prize' 可以生成新特征'whether prize occurs':词频大于1的则特征值为1，否则为0

In [11]:
from sklearn.preprocessing import Binarizer
X = [[4],[1],[3],[0]]
binarizer = Binarizer(threshold=2.0)
X_new = binarizer.fit_transform(X)
print(X_new)

[[1]
 [0]
 [1]
 [0]]


### Discretization:
通过有限的值，将数值特征转换为类别特征。Binarization可以看着是Discretization的特例，比如年龄段特征

### Interaction:
包括相加、相乘、2个数值特征的任意运算、2个分类特征的联合约束。例如，
    
    ’每周访问量‘和’每周销量‘可以生成‘每次访问的销量；
    ‘兴趣’和‘职业’——> ‘兴趣和职业‘，其值如’engineer interested in sports‘

### Polynomial transformation多项式变换：
特征a、b，可以生成二次多项式特征a^2, ab, b^2

In [12]:
from sklearn.preprocessing import PolynomialFeatures
X = [[2, 4],
     [1, 3],
     [3, 2],
     [0, 3]]
poly = PolynomialFeatures(degree=2)
X_new = poly.fit_transform(X)
print(X_new)

[[  1.   2.   4.   4.   8.  16.]
 [  1.   1.   3.   1.   3.   9.]
 [  1.   3.   2.   9.   6.   4.]
 [  1.   0.   3.   0.   0.   9.]]


#### 注意，新特征中包括 1 ——bias or intercept

## 记录每个特征如何生成
这貌似不重要，但我们常常忘记一个特征从何而来。模型训练失败，或者想要创建新特征来提升模型性能时，我们经常要回溯。为了删除无效的特征，添加有效特征，我们必须要理清what and how。

# 3. 算法训练、评估、选择
拿到一个机器学习问题，一个问题通常是：解决它的最好算法是什么？NO free lunch.

我们需要尝试多种方法和调优，才能找出最佳方案。

## 从简单的开始
由于一个算法有多个参数需要调整，穷举法非常耗时，且计算开销很大。NB、LR、SVM 3种算法可以择一作为出发点，具体选择方针如下：

    * 训练集大小
    * 数据集维度
    * 数据是否线性可分
    * 特征是否独立
    * bias 和 variance 的tolerance 和tradeoff
    * 是否有online learning需求

### Naive Bayes:

    一个很简单的算法。对于特征独立、相对小的训练集，表现很好。
    至于大数据集，因为不管真实情况如何，此时也能够假设其特征独立，所以表现也不错。
    因为计算简单，NB的训练通常比其他算法快，但这可能导致高偏差、低方差。
### Logistic Regression

    分类问题中，应用最广的算法。也是所有初试中的首选。
    数据线性可分或近似可分时，表现很好。
    即是非线性可分，或能转换为线性可分，再用LR。
    通过SGD优化方式，LR对大数据集极具扩展性，因此解决大数据问题可以很高效，同时支持在线学习。
    LR 是低偏差、高方差算法，但我们可以添加正则项防止过拟合，如L1、L2、及2者混合。
### SVM
    
    线性可分或不可分都适用。
    线性可分，采用线性核可比肩LR。
    线性不可分，可用非线性核，如RBF。
    对于高维数据集，SVM也能表现得很好，LR得让步于SVM，例如新闻分类有上万维。
    只要选对kernel和参数，SVM能实现很高的accuracy，但计算开销非常大。
    
### 随机森林（决策树）
    
    不在乎数据的线性可分性。
    类别特征无需编码，直接可用。
    易用性和可解释性很好。
    随机森林是决策树的加强，但过拟合风险也加强。
    性能可比SVM，但调参比SVM和神经网络简单很多。
  

### 降低过拟合

    * 交叉验证
    * 正则化
    * 尽可能简单。复杂模型包括树算法的深度过深；线性回归的高项次多项式转换；复杂kernel的SVM。
    * 集成学习

### 诊断过拟合及欠拟合
学习曲线常用于评估模型的bias 和 variance:
理想状态：![ideal](learning_curve_ideal.png)
训练评分接近所求，测试评分也逐渐接近

过拟合： ![overfitting](learning_curve_overfitting.png)
训练得很好，测试不好

欠拟合：![underfitting](learning_curve_underfitting.png)
训练不好，测试也不好

# 4.部署和监控
保存结果模型，并将它们部署到新的数据上，以及监视性能，定期更新预测模型

## 保存、加载、复用模型：

In [14]:
from sklearn import datasets
dataset = datasets.load_diabetes()
X, y = dataset.data, dataset.target

num_new = 30
X_train = X[:-num_new,:]
y_train = y[:-num_new]
X_new = X[-num_new:]
y_new = y[-num_new:]

# Data pre-processing
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train)

import pickle
# Save the scaler
pickle.dump(scaler, open('scaler.p', 'wb'))

X_scaled_train = scaler.transform(X_train)

# Regression model training
from sklearn.svm import SVR
regressor = SVR(C=20)
regressor.fit(X_scaled_train, y_train)

# Save the Regressor
pickle.dump(regressor, open('regressor.p', 'wb'))

Deployment:

In [16]:
my_scaler = pickle.load(open('scaler.p','rb'))
my_regressor = pickle.load(open('regressor.p','rb'))

X_scaled_new = my_scaler.transform(X_new)
predictions = my_regressor.predict(X_scaled_new)

## 监控
现在机器学习系统已上线运行，为了确保一切正常，我们需要定期检测：除了实时预测的情况，同时也要记录ground truth

In [17]:
from sklearn.metrics import r2_score
print('Health check on the model, R^2: {0:.3f}'.format(r2_score(y_new, predictions)))

Health check on the model, R^2: 0.613


### 定期更新模型：
如果性能越来越差，数据的模式就会发生变化。我们可以更新模型。这取决于这个模型能否在线学习，该模型可以用新数据在线更新或重新训练。